In [25]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import re


# Unified data directories
Datadir = r'C:\Users\Administrator\Documents\MnewData'
output_path = r'C:\Users\Administrator\PYMo\Data'

def list_csv_files(directory, keyword):
    """List CSV files in the specified directory that contain the keyword."""
    return [file for file in os.listdir(directory) if keyword in file and file.endswith('.csv')]

def read_and_process_files(directory, keyword, columns=None,encoding='gbk'):
    """Read and process CSV files, returning a concatenated DataFrame."""
    files = list_csv_files(directory, keyword)
    dfs = []
    for file in files:
        file_path = os.path.join(directory, file)
        try:
            df = pd.read_csv(file_path, encoding=encoding, usecols=columns) if columns else pd.read_csv(file_path, encoding=encoding,skiprows=2, header=0, na_values=["n/a", "na", "-"])
            df.columns = df.columns.str.replace(' ', '')  # 去除列名中的空格
            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file}: {e}")
    return pd.concat(dfs, ignore_index=True).drop_duplicates() if dfs else pd.DataFrame()





# Antenna Planning    df_AP

In [26]:
# 读取数据并重命名列
df_AP = read_and_process_files(Datadir, '天线安装规划', ['网元标识', '远端射频单元编号', '本地小区标识1'])

# 重命名列
df_AP.rename(columns={
    '远端射频单元编号': '射频单元编号',
    '本地小区标识1': '小区本地ID'
}, inplace=True)

In [27]:
df_AP.head(10)

,网元标识,射频单元编号,小区本地ID
0,6355679,1,2
4,6355679,0,1
8,6355947,3,4
9,6355947,1,2
16,6355947,0,1
20,6355947,2,3
24,6355826,1,5
28,6355826,4,2
32,6355826,0,4
36,6355826,3,1


# Radio Frequency Planning  df_RF

In [28]:

df_RF = read_and_process_files(Datadir, '射频单元规划', ['网元标识', '射频单元编号', '射频单元RRU安装经度', '射频单元RRU安装纬度'])
# 重命名列
df_RF.rename(columns={
    '射频单元RRU安装经度': 'Longitude',
    '射频单元RRU安装纬度': 'Latitude'
}, inplace=True)


In [29]:
df_RF.head(10)

,网元标识,射频单元编号,Longitude,Latitude
0,6355688,2,114.259369,29.931835
1,6355688,0,114.259369,29.931835
2,6355688,1,114.259369,29.931835
3,6337869,1,110.546023,32.064548
4,6337869,2,110.546023,32.064548
5,6337869,0,110.546023,32.064548
6,6355693,2,114.087013,29.536091
7,6355693,1,114.087013,29.536091
8,6355693,0,114.087013,29.536091
9,6334748,1,110.080330,33.169849


# Cell parameters df_CP

In [30]:
df_CP = read_and_process_files(Datadir, 'NR小区', ['网元标识', '网元名称', '小区本地ID', '小区友好名', 'Nr小区工作频段'])
df_CP.rename(columns={
    '网元名称': 'BBU机房',
    '小区友好名': '小区名称',
    'Nr小区工作频段': '工作频段'
}, inplace=True)
df_CP['工作频段'] = df_CP['工作频段'].str.split('(').str[0]
df_CP['基站名称'] = df_CP['小区名称'].str.replace(r'(-26.*|-07.*)', '', regex=True)

In [31]:
df_CP.head(10)

,网元标识,BBU机房,小区本地ID,小区名称,工作频段,基站名称
0,6355688,咸安胡翰林700M-D5H,2,咸安胡翰林700M-D5H-0712,band28,咸安胡翰林700M-D5H
1,6355688,咸安胡翰林700M-D5H,3,咸安胡翰林700M-D5H-0713,band28,咸安胡翰林700M-D5H
2,6355688,咸安胡翰林700M-D5H,1,咸安胡翰林700M-D5H-0711,band28,咸安胡翰林700M-D5H
3,6337869,房县化龙镇700M-D5H,1,房县化龙镇700M-D5H-0711,band28,房县化龙镇700M-D5H
4,6337869,房县化龙镇700M-D5H,2,房县化龙镇700M-D5H-0712,band28,房县化龙镇700M-D5H
5,6337869,房县化龙镇700M-D5H,3,房县化龙镇700M-D5H-0713,band28,房县化龙镇700M-D5H
6,6355693,咸安黄铁700M-D5H,1,咸安黄铁700M-D5H-0711,band28,咸安黄铁700M-D5H
7,6355693,咸安黄铁700M-D5H,3,咸安黄铁700M-D5H-0713,band28,咸安黄铁700M-D5H
8,6355693,咸安黄铁700M-D5H,2,咸安黄铁700M-D5H-0712,band28,咸安黄铁700M-D5H
9,6334748,郧西二天门700M-D5H,1,郧西二天门700M-D5H-0711,band28,郧西二天门700M-D5H


# Read Map

In [32]:
def read_hubei_map_files(directory, filename):
    """Read the Hubei map file."""
    file_path = os.path.join(directory, filename)
    return gpd.read_file(file_path)

In [33]:
gdffile = '湖北省村级边界.geojson'
gdf = read_hubei_map_files(Datadir, gdffile)

In [34]:
def process_rf_ap_cp(df_ap, df_rf, df_cp, gdf):
    """
    合并天线规划数据、射频单元规划数据和小区规划数据，并与地理数据进行空间连接。

    参数:
    df_ap (pd.DataFrame): 天线规划数据。
    df_rf (pd.DataFrame): 射频单元规划数据。
    df_cp (pd.DataFrame): 小区规划数据。
    gdf (gpd.GeoDataFrame): 地理数据。

    返回值:
    gpd.GeoDataFrame: 合并后的GeoDataFrame，包含规整后的列名。
    """
    # 合并天线规划数据和射频单元规划数据
    df_rfap = pd.merge(df_ap, df_rf, how='inner', 
                       left_on=['网元标识', '射频单元编号'],
                       right_on=['网元标识', '射频单元编号'],
                       suffixes=('_df_AP', '_df_RF'))

    # 选择并重命名列
    df_rfap = df_rfap[['网元标识', '小区本地ID', '射频单元编号', 'Longitude', 'Latitude']]

    # 合并射频单元规划数据和小区规划数据
    df_rac = pd.merge(df_rfap, df_cp, how='inner', 
                      left_on=['网元标识', '小区本地ID'],
                      right_on=['网元标识', '小区本地ID'],
                      suffixes=('_df_RFAP', '_df_CP'))

    # 选择并重命名列
    df_rac = df_rac[['网元标识', '小区本地ID',  'BBU机房','基站名称', '小区名称','工作频段', 'Longitude', 'Latitude']]

    # 创建几何图形列表（点）
    geometry = [Point(xy) for xy in zip(df_rac['Longitude'], df_rac['Latitude'])]

    # 将 pandas DataFrame 转换为 GeoDataFrame
    df_rac = gpd.GeoDataFrame(df_rac, geometry=geometry)

    # 设置 CRS
    if df_rac.crs is None:
        df_rac = df_rac.set_crs(gdf.crs)

    # 空间连接
    gdf_rac = gpd.sjoin(df_rac, gdf, how='inner', predicate='within')

    # 确保 '网元标识' 和 '小区本地ID' 列都是字符串类型
    gdf_rac['网元标识'] = gdf_rac['网元标识'].astype(str)
    gdf_rac['小区本地ID'] = gdf_rac['小区本地ID'].astype(str)

    # 增加列ID
    gdf_rac['ID'] = gdf_rac['网元标识'] + '_' + gdf_rac['小区本地ID']

    # 选择并重命名列
    gdf_rac = gdf_rac[['ID','网元标识', '小区本地ID', 'BBU机房','基站名称', '小区名称','工作频段', 'Longitude', 'Latitude', 'SJGZQYMC', 'DSJGZQYMC', 'QXJGZQYMC', 'XZJGZQYMC', 'CJGZQYMC']].rename(
        columns={'SJGZQYMC': '省份', 'DSJGZQYMC': '地市', 'QXJGZQYMC': '县区', 'XZJGZQYMC': '镇区', 'CJGZQYMC': '村区'}
    )

    return gdf_rac

# 使用示例

gdf_RAC = process_rf_ap_cp(df_AP, df_RF, df_CP, gdf)

In [35]:
gdf_RAC.head(10)

,ID,网元标识,小区本地ID,BBU机房,基站名称,小区名称,工作频段,Longitude,Latitude,省份,地市,县区,镇区,村区
0,6355679_2,6355679,2,通山富池700M-D5H,通山富池700M-D5H,通山富池700M-D5H-0712,band28,114.291809,29.575943,湖北省,咸宁市,崇阳县,路口镇,泉口村
1,6355679_1,6355679,1,通山富池700M-D5H,通山富池700M-D5H,通山富池700M-D5H-0711,band28,114.291809,29.575943,湖北省,咸宁市,崇阳县,路口镇,泉口村
2,6355947_4,6355947,4,通山车田村700M-D5H,通山洪港村五组700M-D5H,通山洪港村五组700M-D5H-0711,band28,114.845510,29.507170,湖北省,咸宁市,通山县,洪港镇,毛田河村
3,6355947_2,6355947,2,通山车田村700M-D5H,通山车田村700M-D5H,通山车田村700M-D5H-0712,band28,114.828488,29.500281,湖北省,咸宁市,通山县,洪港镇,车田村
4,6355947_1,6355947,1,通山车田村700M-D5H,通山车田村700M-D5H,通山车田村700M-D5H-0711,band28,114.828488,29.500281,湖北省,咸宁市,通山县,洪港镇,车田村
5,6355947_3,6355947,3,通山车田村700M-D5H,通山车田村700M-D5H,通山车田村700M-D5H-0713,band28,114.828488,29.500281,湖北省,咸宁市,通山县,洪港镇,车田村
6,6355826_5,6355826,5,崇阳叶门700M-D5H,崇阳叶门700M-D5H,崇阳叶门700M-D5H-0712,band28,113.905539,29.490531,湖北省,咸宁市,崇阳县,石城镇,方山村
7,6355826_2,6355826,2,崇阳叶门700M-D5H,崇阳荻洲700M-D5H,崇阳荻洲700M-D5H-0712,band28,113.963381,29.488554,湖北省,咸宁市,崇阳县,石城镇,荻洲村
8,6355826_4,6355826,4,崇阳叶门700M-D5H,崇阳叶门700M-D5H,崇阳叶门700M-D5H-0711,band28,113.905539,29.490531,湖北省,咸宁市,崇阳县,石城镇,方山村
9,6355826_1,6355826,1,崇阳叶门700M-D5H,崇阳荻洲700M-D5H,崇阳荻洲700M-D5H-0711,band28,113.963381,29.488554,湖北省,咸宁市,崇阳县,石城镇,荻洲村


# Read KPI file and Regular format  df_KPI

In [47]:
df_kpi = read_and_process_files(Datadir, 'DT_PowerBI指标通报计数器_')

def process_df_kpi(df):
    # 提取 NB 和 nrCellCfg 并创建 ID 列
    df[['NB', 'nrCellCfg']] = df['对象'].str.extract(r'gNB=(\d+),nrCellCfg=(\d+)')
    df['ID'] = df['NB'] + '_' + df['nrCellCfg']
    
    # 重新排列列顺序
    new_order = ['ID', 'NB', 'nrCellCfg'] + [col for col in df.columns if col not in ['ID', 'NB', 'nrCellCfg']]
    df = df[new_order].copy()  # 使用 .copy() 创建一个新的 DataFrame 副本

    # 提取括号中的内容作为新的列名
    df.columns = [re.search(r'\((.*?)\)', col).group(1) if re.search(r'\((.*?)\)', col) else col for col in df.columns]

    # 删除指定的列
    columns_to_drop = ['对象', 'Nr小区工作频段', 'MHz', '逻辑小区id']
    df.drop(columns=columns_to_drop, inplace=True)

    # 转换开始时间和结束时间为日期格式
    df.loc[:, '开始时间'] = pd.to_datetime(df['开始时间']).dt.date
    df.loc[:, '结束时间'] = pd.to_datetime(df['结束时间']).dt.date

    # 将 R 开头的列转换为整数类型
    r_columns = [col for col in df.columns if col.startswith('R')]
    df.loc[:, r_columns] = df[r_columns].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

    # 将 K 开头的列转换为带两位小数的浮点类型
    k_columns = [col for col in df.columns if col.startswith('K')]
    df.loc[:, k_columns] = df[k_columns].apply(pd.to_numeric, errors='coerce').fillna(0).round(2)

    return df

df_KPI = process_df_kpi(df_kpi)


In [48]:
df_KPI.head(10)

,ID,NB,nrCellCfg,开始时间,结束时间,R2035_003,R2035_013,R2035_026,R2007_001,R2007_002,...,R1001_001,R1001_004,R1001_007,R1001_008,R1001_012,R1001_015,R1001_018,R1001_019,K1009_001,K1009_002
0,6299318_1,6299318,1,2024-10-01,2024-10-02,12.0,12.0,232.0,21071.0,21043.0,...,47654.0,10985.0,318.0,0.0,47409.0,10966.0,315.0,0.0,18.52,1.07
1,6299318_1,6299318,1,2024-10-02,2024-10-03,20.0,20.0,583.0,62210.0,62081.0,...,96563.0,21972.0,674.0,0.0,95554.0,21915.0,673.0,0.0,29.83,1.88
2,6299318_1,6299318,1,2024-10-03,2024-10-04,30.0,29.0,709.0,73361.0,73156.0,...,111052.0,26896.0,755.0,0.0,110232.0,26834.0,746.0,0.0,39.98,2.34
3,6299318_1,6299318,1,2024-10-04,2024-10-05,20.0,19.0,536.0,56781.0,56673.0,...,92325.0,21638.0,606.0,0.0,91646.0,21585.0,603.0,0.0,29.06,1.62
4,6299318_1,6299318,1,2024-10-05,2024-10-06,13.0,13.0,397.0,44852.0,44738.0,...,74425.0,17539.0,500.0,0.0,73916.0,17493.0,499.0,0.0,20.91,1.42
5,6299318_1,6299318,1,2024-10-06,2024-10-07,6.0,6.0,200.0,21420.0,21394.0,...,37589.0,9302.0,377.0,0.0,37362.0,9283.0,377.0,0.0,11.81,0.93
6,6299318_2,6299318,2,2024-10-01,2024-10-02,21.0,21.0,460.0,21807.0,21679.0,...,74621.0,15028.0,560.0,0.0,74221.0,14999.0,558.0,0.0,30.38,1.42
7,6299318_2,6299318,2,2024-10-02,2024-10-03,27.0,27.0,590.0,65412.0,65159.0,...,103271.0,20207.0,830.0,0.0,102220.0,20157.0,824.0,0.0,33.85,1.79
8,6299318_2,6299318,2,2024-10-03,2024-10-04,38.0,38.0,610.0,77036.0,76713.0,...,117944.0,22895.0,892.0,0.0,116880.0,22827.0,884.0,0.0,32.78,2.21
9,6299318_2,6299318,2,2024-10-04,2024-10-05,29.0,26.0,551.0,59900.0,59714.0,...,102291.0,20786.0,756.0,0.0,101446.0,20730.0,755.0,0.0,28.51,1.45


In [ ]:
def process_df_kpi(df):
    # 提取 NB 和 nrCellCfg 并创建 ID 列
    df[['NB', 'nrCellCfg']] = df['对象'].str.extract(r'gNB=(\d+),nrCellCfg=(\d+)')
    df['ID'] = df['NB'] + '_' + df['nrCellCfg']
    
    # 重新排列列顺序
    new_order = ['ID', 'NB', 'nrCellCfg'] + [col for col in df.columns if col not in ['ID', 'NB', 'nrCellCfg']]
    df = df[new_order]

    # 提取括号中的内容作为新的列名
    df.columns = [re.search(r'\((.*?)\)', col).group(1) if re.search(r'\((.*?)\)', col) else col for col in df.columns]

    # 删除指定的列
    columns_to_drop = ['对象', 'Nr小区工作频段', 'MHz', '逻辑小区id']
    df.drop(columns=columns_to_drop, inplace=True)

    # 转换开始时间和结束时间为日期格式
    df['开始时间'] = pd.to_datetime(df['开始时间']).dt.date
    df['结束时间'] = pd.to_datetime(df['结束时间']).dt.date

    # 将 R 开头的列转换为整数类型
    r_columns = [col for col in df.columns if col.startswith('R')]
    df[r_columns] = df[r_columns].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

    # 将 K 开头的列转换为带两位小数的浮点类型
    k_columns = [col for col in df.columns if col.startswith('K')]
    df[k_columns] = df[k_columns].apply(pd.to_numeric, errors='coerce').fillna(0).round(2)

    return df


,ID,NB,nrCellCfg,开始时间,结束时间,R2035_003,R2035_013,R2035_026,R2007_001,R2007_002,...,R1001_001,R1001_004,R1001_007,R1001_008,R1001_012,R1001_015,R1001_018,R1001_019,K1009_001,K1009_002
401773,6334840_5,6334840,5,2024-10-01,2024-10-02,1,1,567,14585,14572,...,64291,19631,1033,0,64238,19619,1033,0,38.30,4.06
218208,6334723_3,6334723,3,2024-10-27,2024-10-28,12,12,51,400,400,...,4917,1073,16,1,4912,1072,16,1,7.37,1.63
115291,6347215_2,6347215,2,2024-10-10,2024-10-11,1,1,3,20,20,...,5218,1526,52,0,5201,1525,51,0,0.22,0.00
78314,6322850_1,6322850,1,2024-10-07,2024-10-08,14,14,69,1218,1217,...,8567,2826,78,0,8553,2823,78,0,5.63,0.29
239535,6341804_1,6341804,1,2024-10-25,2024-10-26,0,0,7,8,8,...,1094,369,0,0,1094,369,0,0,0.22,0.02
209338,6313227_4,6313227,4,2024-10-26,2024-10-27,0,0,56,138,137,...,6854,1779,84,0,6848,1779,84,0,3.48,0.11
250232,6355469_3,6355469,3,2024-10-26,2024-10-27,1,1,113,1531,1531,...,10847,3248,40,0,10838,3246,40,0,3.67,0.21
550089,6337374_1,6337374,1,2024-11-06,2024-11-07,0,0,20,767,767,...,2485,1394,28,0,2480,1392,28,0,1.46,0.07
420733,6311511_1,6311511,1,2024-10-13,2024-10-14,1,1,140,692,692,...,44749,15726,709,0,44598,15711,706,0,9.72,0.54
207550,6313198_1,6313198,1,2024-10-23,2024-10-24,0,0,15,1269,1269,...,2263,721,36,0,2257,719,36,0,2.22,0.01


In [43]:
df_KPI.head(10)

,ID,NB,nrCellCfg,对象,开始时间,结束时间,Nr小区工作频段,MHz,逻辑小区id,R2035_003,...,R1001_001,R1001_004,R1001_007,R1001_008,R1001_012,R1001_015,R1001_018,R1001_019,K1009_001,K1009_002
0,6299318_1,6299318,1,"丹江蔡湾700M-D5H-0711(gNB=6299318,nrCellCfg=1)",2024-10-01 00:00:00,2024-10-02 00:00:00,band28(758~803MHz)(28),30(5),1.0,12.0,...,47654.0,10985.0,318.0,0.0,47409.0,10966.0,315.0,0.0,18.52,1.07
1,6299318_1,6299318,1,"丹江蔡湾700M-D5H-0711(gNB=6299318,nrCellCfg=1)",2024-10-02 00:00:00,2024-10-03 00:00:00,band28(758~803MHz)(28),30(5),1.0,20.0,...,96563.0,21972.0,674.0,0.0,95554.0,21915.0,673.0,0.0,29.83,1.88
2,6299318_1,6299318,1,"丹江蔡湾700M-D5H-0711(gNB=6299318,nrCellCfg=1)",2024-10-03 00:00:00,2024-10-04 00:00:00,band28(758~803MHz)(28),30(5),1.0,30.0,...,111052.0,26896.0,755.0,0.0,110232.0,26834.0,746.0,0.0,39.98,2.34
3,6299318_1,6299318,1,"丹江蔡湾700M-D5H-0711(gNB=6299318,nrCellCfg=1)",2024-10-04 00:00:00,2024-10-05 00:00:00,band28(758~803MHz)(28),30(5),1.0,20.0,...,92325.0,21638.0,606.0,0.0,91646.0,21585.0,603.0,0.0,29.06,1.62
4,6299318_1,6299318,1,"丹江蔡湾700M-D5H-0711(gNB=6299318,nrCellCfg=1)",2024-10-05 00:00:00,2024-10-06 00:00:00,band28(758~803MHz)(28),30(5),1.0,13.0,...,74425.0,17539.0,500.0,0.0,73916.0,17493.0,499.0,0.0,20.91,1.42
5,6299318_1,6299318,1,"丹江蔡湾700M-D5H-0711(gNB=6299318,nrCellCfg=1)",2024-10-06 00:00:00,2024-10-07 00:00:00,band28(758~803MHz)(28),30(5),1.0,6.0,...,37589.0,9302.0,377.0,0.0,37362.0,9283.0,377.0,0.0,11.81,0.93
6,6299318_2,6299318,2,"丹江蔡湾700M-D5H-0712(gNB=6299318,nrCellCfg=2)",2024-10-01 00:00:00,2024-10-02 00:00:00,band28(758~803MHz)(28),30(5),2.0,21.0,...,74621.0,15028.0,560.0,0.0,74221.0,14999.0,558.0,0.0,30.38,1.42
7,6299318_2,6299318,2,"丹江蔡湾700M-D5H-0712(gNB=6299318,nrCellCfg=2)",2024-10-02 00:00:00,2024-10-03 00:00:00,band28(758~803MHz)(28),30(5),2.0,27.0,...,103271.0,20207.0,830.0,0.0,102220.0,20157.0,824.0,0.0,33.85,1.79
8,6299318_2,6299318,2,"丹江蔡湾700M-D5H-0712(gNB=6299318,nrCellCfg=2)",2024-10-03 00:00:00,2024-10-04 00:00:00,band28(758~803MHz)(28),30(5),2.0,38.0,...,117944.0,22895.0,892.0,0.0,116880.0,22827.0,884.0,0.0,32.78,2.21
9,6299318_2,6299318,2,"丹江蔡湾700M-D5H-0712(gNB=6299318,nrCellCfg=2)",2024-10-04 00:00:00,2024-10-05 00:00:00,band28(758~803MHz)(28),30(5),2.0,29.0,...,102291.0,20786.0,756.0,0.0,101446.0,20730.0,755.0,0.0,28.51,1.45
